<a href="https://colab.research.google.com/github/pthengtr/kcw-analytics/blob/main/notebooks/01_supabase_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")

Loaded: raw_inventory_hq_2024.csv -> (4983, 8)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2881, 49)
Loaded: raw_syp_pidet_purchase_lines.csv -> (26919, 41)
Loaded: raw_syp_sidet_sales_lines.csv -> (35191, 38)
Loaded: raw_syp_simas_sales_bills.csv -> (11923, 49)
Loaded: raw_hq_pimas_purchase_bills.csv -> (83416, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (248835, 41)
Loaded: raw_hq_icmas_products.csv -> (114874, 94)
Loaded: raw_hq_sidet_sales_lines.csv -> (1200038, 38)
Loaded: raw_hq_simas_sales_bills.csv -> (486118, 49)


In [ ]:
BRANCH = 'hq'

In [ ]:
df_sidet = data[f'raw_{BRANCH}_sidet_sales_lines.csv']
df_simas = data[f'raw_{BRANCH}_simas_sales_bills.csv']
df_pidet = data[f'raw_{BRANCH}_pidet_purchase_lines.csv']
df_pimas = data[f'raw_{BRANCH}_pimas_purchase_bills.csv']
df_icmas = data[f'raw_hq_icmas_products.csv']

df_sidet = filter_last_year_from_latest(df_sidet, "BILLDATE")
df_simas = filter_last_year_from_latest(df_simas, "BILLDATE")
df_pidet = filter_last_year_from_latest(df_pidet, "BILLDATE")
df_pimas = filter_last_year_from_latest(df_pimas, "BILLDATE")

print(len(df_sidet))
print(len(df_simas))
print(len(df_pidet))
print(len(df_pimas))
print(len(df_icmas))


193201
61913
39081
10846
114874


In [ ]:

import psycopg2

conn = psycopg2.connect(
    host="aws-0-ap-southeast-1.pooler.supabase.com",   # paste yours here
    port=5432,
    dbname="postgres",
    user="postgres.jdzitzsucntqbjvwiwxm",        # note the dot + project ref
    password="3h3aAixsyK4X762r",
    sslmode="require",
)
print("Connected via pooler ✅")


Connected via pooler ✅


In [ ]:
# Example: upsert product master
result = bulk_upsert_via_stage(
    conn,
    df_sidet,
    target_table=f"raw_{BRANCH}_sidet_lines",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 193201, 'target_table': 'public.raw_hq_sidet_lines', 'stage_table': 'public.raw_hq_sidet_lines_stage'}


In [ ]:
# Example: upsert product master
result = bulk_upsert_via_stage(
    conn,
    df_pidet,
    target_table=f"raw_{BRANCH}_pidet_lines",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 39081, 'target_table': 'public.raw_hq_pidet_lines', 'stage_table': 'public.raw_hq_pidet_lines_stage'}


In [ ]:
# Example: upsert product master
result = bulk_upsert_via_stage(
    conn,
    df_simas,
    target_table=f"raw_{BRANCH}_simas_bills",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 61913, 'target_table': 'public.raw_hq_simas_bills', 'stage_table': 'public.raw_hq_simas_bills_stage'}


In [ ]:
# Example: upsert product master
result = bulk_upsert_via_stage(
    conn,
    df_pimas,
    target_table=f"raw_{BRANCH}_pimas_bills",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 10846, 'target_table': 'public.raw_hq_pimas_bills', 'stage_table': 'public.raw_hq_pimas_bills_stage'}


In [ ]:
# Example: upsert product master
result = bulk_upsert_via_stage(
    conn,
    df_icmas,
    target_table=f"raw_{BRANCH}_icmas_products",
    pk_cols=["ID"],
)

print(result)

{'rows_in_df': 114874, 'target_table': 'public.raw_hq_icmas_products', 'stage_table': 'public.raw_hq_icmas_products_stage'}
